In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.compose import ColumnTransformer

import joblib

import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
cwd = os.getcwd()

In [3]:
parent_dir = os.path.dirname(cwd)

In [6]:
trained_pipeline = joblib.load(f'{parent_dir}/FeatureSelection/SVM_OpenSmile_Rec_Only.pkl')

In [9]:
import opensmile
from sklearn.base import BaseEstimator, TransformerMixin

In [15]:
class FeatureExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.smile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.eGeMAPSv02,
        feature_level=opensmile.FeatureLevel.Functionals,
    )

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Extract features for each audio file and convert to DataFrame
        features_list = [self.smile.process_file(file_path) for file_path in X]
        # Concatenate all extracted features into a single DataFrame
        features_df = pd.concat(features_list, axis=0, ignore_index=True)
        return features_df

In [16]:
new_pipeline = Pipeline([
    ('features', FeatureExtractor()),  # Step 1: Extract features
    ('existing', trained_pipeline)     # Step 2: Use the already trained pipeline
])

In [22]:
audio_dir = f'{parent_dir}/Audio/'
test_audio = [f'{audio_dir}/Atrophy-0A00e04.wav', f'{audio_dir}/Laryngeal cancer-0702gr9.wav']

In [23]:
y_pred = new_pipeline.predict(test_audio)

In [24]:
y_pred

array(['Benign', 'Malignant'], dtype=object)